In [1]:
import sys,os,time
sys.path.append(r'/root/ember')
sys.path.append(r'/root/boosting-ember')
import ember
import pandas as pd
import numpy as np
import lightgbm as lgb
from boostember.features_extended import *

datasetpath = '/root/dataset/ember2018/'
ember.create_vectorized_features(datasetpath, 2)
X_train, y_train, X_test, y_test = ember.read_vectorized_features(datasetpath)

Vectorizing training set


100%|██████████| 800000/800000 [09:06<00:00, 1462.57it/s]


Vectorizing test set


100%|██████████| 200000/200000 [02:13<00:00, 1493.06it/s]

In [2]:
delunlabel = (y_train != -1)
X_train = pd.DataFrame(X_train[delunlabel], columns=emberfeaturesheader())
y_train = y_train[delunlabel]
X_train.shape, y_train.shape

((600000, 2381), (600000,))

In [ ]:
from sklearn.metrics import (roc_auc_score, make_scorer)
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

params = {
    'boosting_type': ['goss','gdbt','dart'],
    'objective': ['binary'],
    'num_iterations': [500, 1000, 1500],
    'learning_rate': [0.005, 0.05, 0.5],
    'num_leaves': [512, 1024, 2048, 4096],
    'feature_fraction': [0.5, 0.8, 1.0, 1.3],
    'bagging_fraction': [0.5, 0.8, 1.0, 1.3]
}

score = make_scorer(roc_auc_score, max_fpr=5e-3)
model = lgb.LGBMClassifier(boosting_type="gbdt", n_jobs=-1, silent=True)
progressive_cv = TimeSeriesSplit(n_splits=3).split(X_train)
grid = GridSearchCV(estimator=model, cv=progressive_cv, param_grid=params, scoring=score, n_jobs=1, verbose=3)
grid.fit(X_train, y_train)


lgbm_dataset = lgb.Dataset(X_train, y_train)
lgbm_model = lgb.train(grid.best_params_, lgbm_dataset)


Fitting 3 folds for each of 1728 candidates, totalling 5184 fits


/root/anaconda3/envs/fyp/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


In [12]:
import joblib
joblib.dump(lgbm_model, 'default_search_params_11_4.pkl')
#joblib.dump(grid, 'griddefaultparams_11_4.pkl')

['default_search_params_11_4.pkl']

In [13]:
print(grid.best_params_)

{'bagging_fraction': 0.5, 'boosting_type': 'goss', 'feature_fraction': 0.5, 'learning_rate': 0.05, 'num_iterations': 1000, 'num_leaves': 512, 'objective': 'binary'}
